In [1]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
import nltk
import glob
from  nltk import word_tokenize
import nltk

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import os
from collections import Counter
from collections import defaultdict
import gc

In [2]:
file_list = glob.glob(os.path.join('hpac_lower_tokenized/hpac_source/', '*'))

In [3]:
file_list

['hpac_lower_tokenized/hpac_source/10000036',
 'hpac_lower_tokenized/hpac_source/10000965',
 'hpac_lower_tokenized/hpac_source/10001184',
 'hpac_lower_tokenized/hpac_source/10003365',
 'hpac_lower_tokenized/hpac_source/10003781',
 'hpac_lower_tokenized/hpac_source/10004060',
 'hpac_lower_tokenized/hpac_source/10004352',
 'hpac_lower_tokenized/hpac_source/10004463',
 'hpac_lower_tokenized/hpac_source/10004778',
 'hpac_lower_tokenized/hpac_source/10005944',
 'hpac_lower_tokenized/hpac_source/10006107',
 'hpac_lower_tokenized/hpac_source/10007296',
 'hpac_lower_tokenized/hpac_source/10007571',
 'hpac_lower_tokenized/hpac_source/10007686',
 'hpac_lower_tokenized/hpac_source/10007867',
 'hpac_lower_tokenized/hpac_source/10008046',
 'hpac_lower_tokenized/hpac_source/10008706',
 'hpac_lower_tokenized/hpac_source/10010115',
 'hpac_lower_tokenized/hpac_source/10011793',
 'hpac_lower_tokenized/hpac_source/10011829',
 'hpac_lower_tokenized/hpac_source/10012170',
 'hpac_lower_tokenized/hpac_source

In [4]:
#nltk.download('punkt')
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
punct = set(list(string.punctuation))
stop = set(stopwords.words('english') + list(string.punctuation))


In [5]:
# http://carrefax.com/new-blog/2017/11/30/find-most-common-words-in-a-corpus
corpus = []
tmp = []

def remove_from_text(text, dels):
    return [w for w in text.split() if w not in dels]

In [6]:
for i, file_path in enumerate(tqdm(file_list)):
    with open(file_path,'r') as f_input:
        corpus += remove_from_text(f_input.read(), punct)
        f_input.close()

100%|██████████| 36225/36225 [01:21<00:00, 447.16it/s]


In [7]:
vec = CountVectorizer().fit(corpus)
bag_of_words = vec.transform(corpus)

In [8]:
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]

words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
top_1000 = words_freq[:1000]
top_1000

[('the', 20768105),
 ('to', 14167997),
 ('and', 13033255),
 ('he', 9631030),
 ('of', 8115773),
 ('you', 7343775),
 ('was', 7283629),
 ('his', 6744480),
 ('it', 6368322),
 ('that', 6082404),
 ('her', 6030557),
 ('in', 5760043),
 ('she', 5627843),
 ('harry', 4000496),
 ('had', 3978687),
 ('as', 3746535),
 ('with', 3510165),
 ('at', 3407543),
 ('for', 3369579),
 ('him', 3332448),
 ('on', 3016367),
 ('but', 2980326),
 ('be', 2411956),
 ('do', 2333868),
 ('not', 2315770),
 ('they', 2295794),
 ('said', 2262365),
 ('have', 2151138),
 ('is', 2093786),
 ('what', 2089675),
 ('me', 2009174),
 ('up', 1946867),
 ('would', 1904645),
 ('this', 1902339),
 ('did', 1879482),
 ('out', 1832826),
 ('hermione', 1831472),
 ('from', 1765326),
 ('so', 1760331),
 ('my', 1760095),
 ('could', 1688265),
 ('all', 1675154),
 ('were', 1668985),
 ('we', 1617270),
 ('if', 1544022),
 ('just', 1490520),
 ('when', 1405664),
 ('back', 1400309),
 ('one', 1400091),
 ('draco', 1389977),
 ('them', 1382608),
 ('about', 1348980)

In [9]:

clean_corpus = []
for text in corpus:
    clean_corpus += remove_from_text(text, stop_words)


In [10]:
names = pd.read_csv('harry-potter-characters.csv', sep =',',names =['name','prof','c3'])
names = names.iloc[1:]
names.fillna('',inplace = True)
names['len'] = names.name.apply(lambda x: len(x.split()))
names['fnames'] = names.name.str.replace(r'Mrs\.|Mr\.|The','')
names['only_name'] = names.fnames.apply(lambda x: x.strip().split()[0]).str.lower()

surnames = names[names['len']==2].name.str.lower().tolist()

mask =  names.prof.str.contains('(p|P)rofessor') 
professors = names[mask].name.apply(lambda x: 'professor '+x.split()[1] if len(x.split())>1 else 'professor '+x.split()[0]).str.lower().tolist()

/home/u_m1c2d/kernel/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/home/u_m1c2d/kernel/env/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.


In [11]:
list_of_names = {}
only_names = names.only_name.unique()
for k,v in words_freq:
    if k in only_names:
        list_of_names[k] = v

In [12]:
Counter(list_of_names).most_common(10)


[('harry', 4000496),
 ('hermione', 1831472),
 ('draco', 1389977),
 ('ron', 906196),
 ('severus', 655654),
 ('ginny', 639546),
 ('sirius', 630217),
 ('james', 468947),
 ('remus', 433776),
 ('lily', 423882)]

In [13]:
bigrm = nltk.ngrams(clean_corpus,2)

In [14]:
profs = {}
fst_lst_names = {}
for v in tqdm(bigrm):
    value = v[0]+' '+v[1]
    if value in surnames:
        if value in fst_lst_names:
            fst_lst_names[value]+=1
        else:
            fst_lst_names.setdefault(value,1)
    if value in professors:
        if value in profs:
            profs[value]+=1
        else:
            profs.setdefault(value,1)

280752023it [11:10, 418527.43it/s]


In [15]:
Counter(fst_lst_names).most_common(10)

[('harry potter', 133207),
 ('draco malfoy', 49844),
 ('severus snape', 32138),
 ('hermione granger', 31520),
 ('james potter', 28926),
 ('sirius black', 27877),
 ('lucius malfoy', 25039),
 ('albus dumbledore', 22204),
 ('remus lupin', 13896),
 ('ginny weasley', 11291)]

In [16]:
Counter(profs).most_common(10)

[('professor snape', 45477),
 ('professor mcgonagall', 42934),
 ('professor dumbledore', 24122),
 ('professor lupin', 10089),
 ('professor flitwick', 9536),
 ('professor slughorn', 5876),
 ('professor sprout', 5782),
 ('professor trelawney', 3185),
 ('professor vector', 1853),
 ('professor quirrell', 1766)]